In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_excel(r'M:/Risk Management/DW/Scorecard/TUX_Scores/MarchPayloads/Output_Files/March__Final__Gen12and13_wHardcuts_decile.V3.xlsx')

In [3]:
scorecard_thresholds = {
    'Scorecard1': [
        (723, 900, 1),   # Tier 1: 723-900
        (705, 722, 2),   # Tier 2: 705-722
        (689, 704, 3),   # Tier 3: 689-704
        (670, 688, 4),   # Tier 4: 670-688
        (0, 669, 5)      # Decline: 0-669 (mapped to Tier 5 as decline)
    ],
    'Scorecard2': [
        (715, 900, 6),   # Tier 6: 715-900
        (704, 714, 7),   # Tier 7: 704-714
        (0, 703, 8)      # Decline: 0-703 (mapped to Tier 8 as decline)
    ],   
    'Scorecard3': [
        (717, 900, 4),   # Tier 4: 717-900
        (706, 716, 5),   # Tier 5: 706-716
        (697, 705, 6),   # Tier 6: 697-705
        (689, 696, 7),   # Tier 7: 689-696
        (0, 688, 8)      # Decline: 0-688 (mapped to Tier 8 as decline)
    ],
    'Scorecard4': [
        (755, 900, 1),   # Tier 1: 755-900
        (736, 754, 2),   # Tier 2: 736-754
        (721, 735, 3),   # Tier 3: 721-735
        (704, 720, 4),   # Tier 4: 704-720
        (0, 703, 5)      # Decline: 0-703 (mapped to Tier 5 as decline)
    ] 
}


In [4]:
def get_tier(scorecard_name, score):
    # Retrieve the scorecard's thresholds (ranges) and their associated tiers
    tiers = scorecard_thresholds.get(scorecard_name)
    
    # If the scorecard is not found, return 'Decline'
    if not tiers:
        return 'Decline'

    # Loop through each range in the scorecard
    for min_val, max_val, tier in tiers:
        if min_val <= score <= max_val:
            # Return the associated tier
            return tier
    
    # Return 'Decline' if no matching tier is found
    return 'Decline'


In [5]:
# # Function to assign tiers to the DataFrame
# def assign_tiers(df):
#     df['Tier'] = df.apply(lambda row: get_tier(row['CreditScorecardName'], row['AvgEQScore60_40']), axis=1)
#     return df


In [6]:
def assign_tiers(df):
    def convert_declines(tier):
        return 'Decline' if tier == 8 else tier

    df['Tier'] = df.apply(lambda row: convert_declines(get_tier(row['CreditScorecardName'], row['AvgEQScore60_40'])), axis=1)
    return df


In [7]:
def assign_scorecard(df):
    def get_scorecard(row):
        # Situation 2: Assign directly based on the scorecard if it is Scorecard 1 or Scorecard 4
        if row['CreditScorecardName'] == 'Scorecard1' or row['CreditScorecardName'] == 'Scorecard4':
            return row['CreditScorecardName']
        
        # For all other scorecards, a placeholder logic (customize further if needed)
        return 'Best_Fit_Scorecard'

    df['Assigned_Scorecard'] = df.apply(get_scorecard, axis=1)
    return df

In [8]:
# def final_tier_assignment(df):
#     def determine_final_tier(group):
#         tiers = group['Tier'].tolist()
#         scorecards = group['Assigned_Scorecard'].tolist()

#         if 'Decline' in tiers:
#             final_tier = 'Decline'
#         else:
#             if len(group) == 2:  # Dual applicant case
#                 if 'Scorecard1' in scorecards and 'Scorecard4' in scorecards:
#                     # Situation 3: Both scorecards have valid tiers, choose best tier
#                     final_tier = max(tiers)
#                 else:
#                     # Situation 4: Other scorecard combinations, use worst tier
#                     final_tier = min(tiers)
#             else:
#                 # Single applicant, use the row-tier directly
#                 final_tier = max(tiers)  # If only one row, use that row's tier

#         group['Final_Tier'] = final_tier
#         return group

#     # Apply final tier assignment logic after grouping by AccountId
#     df = df.groupby('AccountId').apply(determine_final_tier)
#     return df

In [9]:
def final_tier_assignment(df):
    def determine_final_tier(group):
        tiers = group['Tier'].tolist()
        scorecards = group['Assigned_Scorecard'].tolist()

        if any(t == 'Decline' for t in tiers):
            final_tier = 'Decline'
        else:
            if len(group) == 2:  # Dual applicant case
                if 'Scorecard1' in scorecards and 'Scorecard4' in scorecards:
                    final_tier = max(tiers)
                else:
                    final_tier = min(tiers)
            else:
                final_tier = tiers[0]

        group['Final_Tier'] = final_tier
        return group

    df = df.groupby('AccountId').apply(determine_final_tier)
    return df


In [10]:
def process_df(df):
    df = assign_tiers(df)               # Step 1: Assign tiers
    df = assign_scorecard(df)            # Step 2: Assign scorecards based on business logic
    df = final_tier_assignment(df)      # Step 3: Apply final tier assignment
    return df

In [11]:
# Process the DataFrame
df_processed = process_df(df)

C:\Users\dwolters\AppData\Local\Temp\ipykernel_6852\1912309331.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('AccountId').apply(determine_final_tier)


In [12]:
df_processed['Final_Tier'] = df_processed['Final_Tier'].replace(8, 'Decline')

In [13]:
df_processed.to_excel(r'M:/Risk Management/DW/Scorecard/TUX_Scores/MarchPayloads/Output_Files/March_TierAssigments_v5.xlsx', index=False)